In [1]:
from datasets import load_dataset

dataset = load_dataset("anon8231489123/ShareGPT_Vicuna_unfiltered")
print(dataset["train"][0])  # Inspect the first conversation

README.md:   0%|          | 0.00/4.41k [00:00<?, ?B/s]

DataFilesNotFoundError: No (supported) data files found in anon8231489123/ShareGPT_Vicuna_unfiltered

In [2]:
from datasets import load_dataset

dataset = load_dataset("OpenAssistant/oasst1")
print(b  # Inspect the first conversation

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

(…)-00000-of-00001-b42a775f407cee45.parquet:   0%|          | 0.00/39.5M [00:00<?, ?B/s]

(…)-00000-of-00001-134b8fd0c89408b6.parquet:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

{'message_id': '6ab24d72-0181-4594-a9cd-deaf170242fb', 'parent_id': None, 'user_id': 'c3fe8c76-fc30-4fa7-b7f8-c492f5967d18', 'created_date': '2023-02-05T14:23:50.983374+00:00', 'text': 'Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.', 'role': 'prompter', 'lang': 'en', 'review_count': 3, 'review_result': True, 'deleted': False, 'rank': None, 'synthetic': False, 'model_name': None, 'detoxify': {'toxicity': 0.00044308538781479, 'severe_toxicity': 3.252684837207198e-05, 'obscene': 0.00023475120542570949, 'identity_attack': 0.0001416115992469713, 'insult': 0.00039489680784754455, 'threat': 4.075629112776369e-05, 'sexual_explicit': 2.712695459194947e-05}, 'message_tree_id': '6ab24d72-0181-4594-a9cd-deaf170242fb', 'tree_state': 'ready_for_export', 'emojis': {'name': ['+1', '_skip_reply', '_skip_ranking'], 'count': [10, 1, 4]}, 'labels': {'name': [

In [8]:
import json
dataset["train"][0]

{'message_id': '6ab24d72-0181-4594-a9cd-deaf170242fb',
 'parent_id': None,
 'user_id': 'c3fe8c76-fc30-4fa7-b7f8-c492f5967d18',
 'created_date': '2023-02-05T14:23:50.983374+00:00',
 'text': 'Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.',
 'role': 'prompter',
 'lang': 'en',
 'review_count': 3,
 'review_result': True,
 'deleted': False,
 'rank': None,
 'synthetic': False,
 'model_name': None,
 'detoxify': {'toxicity': 0.00044308538781479,
  'severe_toxicity': 3.252684837207198e-05,
  'obscene': 0.00023475120542570949,
  'identity_attack': 0.0001416115992469713,
  'insult': 0.00039489680784754455,
  'threat': 4.075629112776369e-05,
  'sexual_explicit': 2.712695459194947e-05},
 'message_tree_id': '6ab24d72-0181-4594-a9cd-deaf170242fb',
 'tree_state': 'ready_for_export',
 'emojis': {'name': ['+1', '_skip_reply', '_skip_ranking'],
  'count': [10

In [21]:
from huggingface_hub import HfApi
from dotenv import load_dotenv
import requests
load_dotenv()

api = HfApi()

def search_datasets(api, search_term, limit=10):
    """Search for datasets using Hugging Face Hub API, filtering for specific conversational formats."""
    datasets = api.list_datasets(search=search_term, limit=limit)
    matching_datasets = []

    for dataset in datasets:
        # Filter based on likely format match in dataset ID or tags
        if "conversation" in dataset.id.lower() or "instruction" in dataset.tags:
            matching_datasets.append(dataset)
            print(f"Name: {dataset.id} - Tags: {dataset.tags}")
    return matching_datasets

def validate_dataset_format_via_api(dataset_name, sample_size=5):
    """Validate dataset format using Hugging Face dataset API by inspecting a sample."""
    try:
        # Fetch a sample from the dataset preview API
        response = requests.get(f"https://huggingface.co/api/datasets/{dataset_name}/first-rows?rows={sample_size}")
        response.raise_for_status()
        data = response.json()["rows"]

        for row in data:
            # Validate each row in the sample
            if not isinstance(row["row"], list):
                return False, f"Row is not a list: {row}"
            for entry in row["row"]:
                if not isinstance(entry, dict):
                    return False, f"Entry is not a dictionary: {entry}"
                if "from" not in entry or "value" not in entry:
                    return False, f"Missing required keys in entry: {entry}"
                if entry["from"] not in ["human", "gpt"]:
                    return False, f"Invalid 'from' value in entry: {entry}"
        return True, "Dataset matches the required format."
    except Exception as e:
        return False, f"Error validating dataset: {e}"

def filter_domain_specific_datasets(api, search_term, domain_keywords, limit=10):
    """Search and validate datasets for specific domains like finance or math."""
    datasets = search_datasets(api, search_term, limit)
    domain_specific_datasets = []

    for dataset in datasets:
        dataset_name = dataset.id
        valid, message = validate_dataset_format_via_api(dataset_name)
        if valid and any(keyword.lower() in dataset_name.lower() for keyword in domain_keywords):
            domain_specific_datasets.append(dataset_name)
            print(f"Valid Domain-Specific Dataset: {dataset_name}")
        else:
            print(f"Dataset {dataset_name} skipped: {message}")

    return domain_specific_datasets

# Search for datasets with likely multi-turn conversational format
search_term = "multi-turn conversation"
domain_keywords = ["finance", "math"]
domain_specific_datasets = filter_domain_specific_datasets(api, search_term, domain_keywords, limit=20)

# Final results
print("\nValidated Domain-Specific Datasets:")
for dataset in domain_specific_datasets:
    print(f"Name: {dataset}")


Name: ThaiSyntheticQA/WangchanThaiInstruct_Multi-turn_Conversation_Dataset - Tags: ['task_categories:text-generation', 'language:th', 'license:cc-by-sa-4.0', 'size_categories:1K<n<10K', 'format:json', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us', 'synthetic', 'instruction-finetuning']
Name: amuvarma/multiturn_conversations_17k - Tags: ['size_categories:10K<n<100K', 'format:parquet', 'modality:tabular', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us']
Name: amuvarma/multiturn_conversations_17k-audio - Tags: ['size_categories:n<1K', 'format:parquet', 'modality:audio', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us']
Name: amuvarma/multiturn_conversations_17k-audio-0 - Tags: ['size_categories:n<1K', 'format:parquet', 'modality:audio', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

(…)haiInstruct_7.24_multi-turn-chat-2.jsonl:   0%|          | 0.00/109M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5014 [00:00<?, ? examples/s]

Dataset ThaiSyntheticQA/WangchanThaiInstruct_Multi-turn_Conversation_Dataset skipped: Row is not a list: {'ID': 'Finance_1003', 'Domain': 'Finance', 'Instruction': 'เพราะเหตุใด เศรษฐกิจญี่ปุ่นเพียงประเทศเดียว ยังคงอยู่ในโหมดการผ่อนคลายนโยบายการเงินแบบเต็มตัว', 'Input': None, 'Output': 'ฮารุอิโกะ คูโรด้า ได้ให้คำตอบว่า มีเหตุผลอยู่ 3 ประการ ได้แก่\n\n\n1. GDP: ขนาดเศรษฐกิจของญี่ปุ่นยังเล็กกว่าช่วงก่อนโควิดอยู่ร้อยละ 2.7 สภาพถือว่าใกล้เคียงกับบ้านเราในขณะนี้ ส่วนขนาดเศรษฐกิจของสหรัฐและยุโรปใหญ่กว่าตอนก่อนโควิดอยู่ร้อยละ 3.7 และ 0.6 ตามลำดับ ไปเรียบร้อยแล้ว ดังนั้น ตรรกะในการคิดออกจะคล้ายกับแบงก์ชาติบ้านเราคือควรผ่อนคลายนโยบายการเงินเพื่อประคองเศรษฐกิจไปก่อน\n\n\n2. Terms of Trade: แม้ว่าอัตราการเติบโต GDP ของญี่ปุ่น ที่ร้อยละ 2.1 เมื่อวัดในปีแบบงบประมาณ 2021 ทว่าหากพิจารณาจาก ตัววัด GNI (Gross National Income)  ซึ่งทำการเพิ่มปัจจัยอื่น ๆ อาทิ รายได้จากการลงทุนในต่างประเทศ และ รายได้จากกำไร หรือขาดทุนจากการค้า ซึ่งมีค่าเป็นลบจากการที่ญี่ปุ่นเป็นผู้นำเข้าสินค้าโภคภัณฑ์ซึ่งมีราคาสูงขึ้นเมื่

...

Generating train split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset amuvarma/multiturn_conversations_17k-audio-2 skipped: Error validating dataset: To support decoding audio files, please install 'librosa' and 'soundfile'.


README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/300M [00:00<?, ?B/s]

KeyboardInterrupt: 

## Junk

In [9]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("mlabonne/FineTome-100k")
df = pd.DataFrame(dataset["train"])
# print(dataset["train"][0])  # Inspect the first conversation

In [13]:
df.sample(1)['conversations'].values[0]

[{'from': 'human',
  'value': 'How can you graph the equation #2y-8x=0# using the slope-intercept form?'},
 {'from': 'gpt',
  'value': 'By isolating #y# in the equation, we get #y=4x#. This is in the slope-intercept form #y=mx+b#, where #m=4# is the slope. The y-intercept is #0#, so the graph will pass through the origin #(0,0)#. Using the slope of #4#, we can find another point by moving up #4# units and right #1# unit from the origin, giving us the point #(1,4)#. Connecting these two points gives us the graph of the equation.'}]

In [20]:
from huggingface_hub import HfApi
from dotenv import load_dotenv
load_dotenv()

api = HfApi()

def search_datasets(api, search_term, limit=10):
    """
    Search for datasets using Hugging Face Hub API, filtering by tags or dataset IDs.
    """
    datasets = api.list_datasets(search=search_term, limit=limit)
    matching_datasets = []

    for dataset in datasets:
        # Filter based on the dataset ID and tags
        if "conversation" in dataset.id.lower() or any("conversation" in tag for tag in dataset.tags):
            matching_datasets.append(dataset)
            print(f"Name: {dataset.id} - Tags: {dataset.tags}")

conversational_datasets = search_datasets(api, search_term="multi-turn conversation", limit=10)

# Search for datasets related to finance
# datasets = api.list_datasets(search="finance", limit=10)
# for dataset in datasets:
#     print(f"Name: {dataset.id} - Description: {dataset.description}")
#     break


Name: ThaiSyntheticQA/WangchanThaiInstruct_Multi-turn_Conversation_Dataset - Tags: ['task_categories:text-generation', 'language:th', 'license:cc-by-sa-4.0', 'size_categories:1K<n<10K', 'format:json', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us', 'synthetic', 'instruction-finetuning']
Name: amuvarma/multiturn_conversations_17k - Tags: ['size_categories:10K<n<100K', 'format:parquet', 'modality:tabular', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us']
Name: amuvarma/multiturn_conversations_17k-audio - Tags: ['size_categories:n<1K', 'format:parquet', 'modality:audio', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us']
Name: amuvarma/multiturn_conversations_17k-audio-0 - Tags: ['size_categories:n<1K', 'format:parquet', 'modality:audio', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant

In [18]:
df.sample(1)['conversations'].values[0]

[{'from': 'human',
  'value': 'Why are electromagnetic waves classified as transverse waves?'},
 {'from': 'gpt',
  'value': 'Electromagnetic waves are classified as transverse waves because their electric and magnetic fields oscillate perpendicular to the direction of wave propagation. In other words, while the wave travels, the magnetic field lines are always at a right angle to the electric field lines. This perpendicular arrangement of the fields distinguishes electromagnetic waves from longitudinal waves, where the vibrations occur along the direction of wave travel. This unique feature of electromagnetic waves makes them suitable for various applications, such as radio communication, light transmission, and X-ray imaging.'}]

In [15]:
dataset = load_dataset("gresearch/financial-question-dataset")
df = dataset["train"][0] # Inspect the first conversation
df.sample(1)['conversations'].values[0]

DatasetNotFoundError: Dataset 'gresearch/financial-question-dataset' doesn't exist on the Hub or cannot be accessed.